## JSSP Tabu Search and Genetic Algorithm Example

This notebook demonstraits how to use both prallel tabu seach and the genetic algorithm.

In [1]:
from JSSP.data import Data
import os

# initialize data
data_directory = '../data/given_data'
Data.initialize_data_from_csv(data_directory + '/sequenceDependencyMatrix.csv',
                              data_directory + '/machineRunSpeed.csv',
                              data_directory + '/jobTasks.csv')

# uncomment this to print the data that was read in
# Data.print_data()

### Tabu Search Before Genetic Algorithm

In [2]:
from JSSP.solver import Solver

output_dir = './example_output'
output_results = False
auto_open = False
verbose = False
benchmark = False
progress_bar = False

# ts parameters
ts_runtime = 30 # in seconds
num_processes = 4
tabu_list_size = 50
neighborhood_size = 200
neighborhood_wait = 0.12
probability_change_machine = 0.8
reset_threshold = 100

# ga parameters
ga_runtime = 30 # in seconds
mutation_probability = 0.8
population_size = 100
selection_size = 6
population = []

# create solver
solver = Solver()

# run tabu seach
solution = solver.tabu_search_time(ts_runtime,
                                   num_processes,
                                   tabu_list_size,
                                   neighborhood_size,
                                   neighborhood_wait,
                                   probability_change_machine,
                                   reset_threshold,
                                   benchmark=benchmark,
                                   verbose=verbose,
                                   progress_bar=progress_bar
                                  )
    
print('Tabu Search Solution:')
solution.pprint()

# add all tabu search solutions to population
population += solver.ts_all_solutions

# run genetic algorithm
solution = solver.genetic_algorithm_time(ga_runtime, 
                                         population, 
                                         population_size, 
                                         mutation_probability, 
                                         selection_size, 
                                         benchmark=benchmark,
                                         verbose=verbose,
                                         progress_bar=progress_bar
                                        )

print('Genetic Algorithm Solution:')
solution.pprint()

if output_results:
    if benchmark:
        solver.output_benchmark_results(output_dir, name='example_benchmark', auto_open=auto_open)
    else:
        solution.create_schedule(output_dir)

Tabu Search Solution:
makespan = 4063.039735099338
machine_makespans = [4027.146341463415, 3995.396551724137, 4042.118421052631, 4062.759036144578, 4050.882882882883, 4049.2899999999995, 4037.3877551020405, 4063.039735099338]
operation_list =
[[   19     0     0     2 23693]
 [   25     1     0     7  6435]
 [   44     1     0     1 10819]
 ...
 [   24     6     4     4 14392]
 [    6     4     2     6  7660]
 [    6     5     2     0  6053]]
Genetic Algorithm Solution:
makespan = 4063.039735099338
machine_makespans = [4027.146341463415, 3995.396551724137, 4042.118421052631, 4062.759036144578, 4050.882882882883, 4049.2899999999995, 4037.3877551020405, 4063.039735099338]
operation_list =
[[   19     0     0     2 23693]
 [   25     1     0     7  6435]
 [   44     1     0     1 10819]
 ...
 [   24     6     4     4 14392]
 [    6     4     2     6  7660]
 [    6     5     2     0  6053]]


### Genetic Algorithm Before Tabu Search

In [3]:
from JSSP.solver import Solver

output_dir = './example_output'
output_results = False
auto_open = False
verbose = False
benchmark = False
progress_bar = False

# ts parameters
ts_runtime = 30 # in seconds
num_processes = 4
tabu_list_size = 50
neighborhood_size = 200
neighborhood_wait = 0.1
probability_change_machine = 0.8
reset_threshold = 100

# ga parameters
ga_runtime = 30 # in seconds
mutation_probability = 0.8
population_size = 200
selection_size = 5
population = []

# create solver
solver = Solver()

# run geneitc algorithm
solution = solver.genetic_algorithm_time(ga_runtime, 
                                         population, 
                                         population_size, 
                                         mutation_probability, 
                                         selection_size, 
                                         benchmark=benchmark,
                                         verbose=verbose,
                                         progress_bar=progress_bar
                                        )

print('Genetic Algorithm Solution:')
solution.pprint()

# pass best solutions found to tabu search
initial_solutions = sorted(solver.ga_result_population)[:num_processes]

# run tabu seach
solution = solver.tabu_search_time(ts_runtime,
                                   num_processes,
                                   tabu_list_size,
                                   neighborhood_size,
                                   neighborhood_wait,
                                   probability_change_machine,
                                   reset_threshold,
                                   initial_solutions=initial_solutions,
                                   benchmark=benchmark,
                                   verbose=verbose,
                                   progress_bar=progress_bar
                                  )
    
print('Tabu Search Solution:')
solution.pprint()

if output_results:
    if benchmark:
        solver.output_benchmark_results(output_dir, name='example_benchmark', auto_open=auto_open)
    else:
        solution.create_schedule(output_dir)

Genetic Algorithm Solution:
makespan = 4691.706896551726
machine_makespans = [4519.196084724005, 4691.706896551726, 4523.815789473685, 4532.084227176618, 4525.365937503615, 4507.670000000001, 4606.693877551019, 4613.890614295502]
operation_list =
[[   17     0     0     7 11056]
 [   27     0     0     4 18694]
 [    3     0     0     6 20912]
 ...
 [   21     4     3     6 12909]
 [   29     3     2     3  6080]
 [   42     2     2     5 17151]]
Tabu Search Solution:
makespan = 4059.1700000000005
machine_makespans = [4048.7398373983738, 3985.068965517241, 4008.4810526315796, 4034.89156626506, 4019.162162162161, 4059.1700000000005, 4054.15306122449, 4056.1456953642387]
operation_list =
[[   21     0     0     7 19015]
 [    8     1     0     2  6671]
 [   16     0     0     0  7276]
 ...
 [   14     6     5     7 19098]
 [    8     2     1     6 10343]
 [   32     5     4     6 16111]]
